# Trying out different middleware integrations

## Summarization Middleware

Just trying out here what is seen from an `astream_events` perspective when summarization occurres in middleware:

In [4]:
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI
from langchain.agents.middleware import SummarizationMiddleware
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.graph import StateGraph, START, END
from langchain.agents import AgentState


agent = create_agent(
        model=ChatOpenAI(model="gpt-4.1"),
        tools=[],
        system_prompt="You are an helpful AI assistant that answers my questions.",
        middleware=[
            SummarizationMiddleware(
                model=ChatOpenAI(model="gpt-4o-mini"),
                max_tokens_before_summary=50,  # Trigger summarization at 20000 tokens
                messages_to_keep=2,  # Keep last 10 messages after summary
            ),
        ]
    )

def agent_node(state: AgentState):
    result = agent.invoke({"messages": state["messages"]})
    return {"messages": result['messages']}

checkpointer = InMemorySaver()

builder = StateGraph(AgentState)
builder.add_node("agent", agent_node)
builder.add_edge(START, "agent")
builder.add_edge("agent", END)
graph = builder.compile(checkpointer=checkpointer)

In [5]:
async for event in graph.astream_events({"messages": [HumanMessage(content="Hello, how are you today?")]}, config={"configurable": {"thread_id": "1"}}):
    print(event)

{'event': 'on_chain_start', 'data': {'input': {'messages': [HumanMessage(content='Hello, how are you today?', additional_kwargs={}, response_metadata={})]}}, 'name': 'LangGraph', 'tags': [], 'run_id': '61bb45ae-b353-4c74-ad8b-ce479a1981dc', 'metadata': {'thread_id': '1'}, 'parent_ids': []}
{'event': 'on_chain_start', 'data': {'input': {'messages': [HumanMessage(content='Hello, how are you today?', additional_kwargs={}, response_metadata={}, id='8b41f96d-1dd3-4d5d-abb0-058c860af988')]}}, 'name': 'agent', 'tags': ['graph:step:1'], 'run_id': '1fbed631-3d2e-4f8b-9bff-8a17feaac6fb', 'metadata': {'thread_id': '1', 'langgraph_step': 1, 'langgraph_node': 'agent', 'langgraph_triggers': ('branch:to:agent',), 'langgraph_path': ('__pregel_pull', 'agent'), 'langgraph_checkpoint_ns': 'agent:2115448e-b7ab-06a7-f012-813364e56967'}, 'parent_ids': ['61bb45ae-b353-4c74-ad8b-ce479a1981dc']}
{'event': 'on_chain_start', 'data': {'input': {'messages': [HumanMessage(content='Hello, how are you today?', additi

In [6]:
async for event in graph.astream_events({"messages": [HumanMessage(content="Tell me a story")]}, config={"configurable": {"thread_id": "1"}}):
    print(event)

{'event': 'on_chain_start', 'data': {'input': {'messages': [HumanMessage(content='Tell me a story', additional_kwargs={}, response_metadata={})]}}, 'name': 'LangGraph', 'tags': [], 'run_id': '9448265a-20d1-45f3-b731-3ae5cc787224', 'metadata': {'thread_id': '1'}, 'parent_ids': []}
{'event': 'on_chain_start', 'data': {'input': {'messages': [HumanMessage(content='Hello, how are you today?', additional_kwargs={}, response_metadata={}, id='8b41f96d-1dd3-4d5d-abb0-058c860af988'), AIMessage(content="Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to help you! How can I assist you today?", additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_433e8c8649', 'service_tier': 'default', 'model_provider': 'openai'}, id='lc_run--315cb9bc-1d9d-4c2d-a67b-62d9cb4d1614', usage_metadata={'input_tokens': 29, 'output_tokens': 30, 'total_tokens': 59, 'input_token_details': {'audio': 0, 'cache_read':

In [7]:
async for event in graph.astream_events({"messages": [HumanMessage(content="What was the story you told me about?")]}, config={"configurable": {"thread_id": "1"}}):
    print(event)

{'event': 'on_chain_start', 'data': {'input': {'messages': [HumanMessage(content='What was the story you told me about?', additional_kwargs={}, response_metadata={})]}}, 'name': 'LangGraph', 'tags': [], 'run_id': '8de8c306-23df-4653-ae5d-378a3564b494', 'metadata': {'thread_id': '1'}, 'parent_ids': []}
{'event': 'on_chain_start', 'data': {'input': {'messages': [HumanMessage(content='Hello, how are you today?', additional_kwargs={}, response_metadata={}, id='8b41f96d-1dd3-4d5d-abb0-058c860af988'), AIMessage(content="Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to help you! How can I assist you today?", additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_433e8c8649', 'service_tier': 'default', 'model_provider': 'openai'}, id='lc_run--315cb9bc-1d9d-4c2d-a67b-62d9cb4d1614', usage_metadata={'input_tokens': 29, 'output_tokens': 30, 'total_tokens': 59, 'input_token_details': {'au

{'event': 'on_chat_model_stream', 'data': {'chunk': AIMessageChunk(content='', additional_kwargs={}, response_metadata={'model_provider': 'openai'}, id='lc_run--6c0e4040-0c8b-4f8c-937a-dd49ba48111d')}, 'run_id': '6c0e4040-0c8b-4f8c-937a-dd49ba48111d', 'name': 'ChatOpenAI', 'tags': ['seq:step:1'], 'metadata': {'thread_id': '1', 'langgraph_step': 1, 'langgraph_node': 'SummarizationMiddleware.before_model', 'langgraph_triggers': ('branch:to:SummarizationMiddleware.before_model',), 'langgraph_path': ('__pregel_pull', 'SummarizationMiddleware.before_model'), 'langgraph_checkpoint_ns': 'agent:ae00444a-c3ff-34b1-640d-e0b69c4fbac7|SummarizationMiddleware.before_model:8737d91d-574f-576d-b2f6-32b09312656c', 'checkpoint_ns': 'agent:ae00444a-c3ff-34b1-640d-e0b69c4fbac7', 'ls_provider': 'openai', 'ls_model_name': 'gpt-4o-mini', 'ls_model_type': 'chat', 'ls_temperature': None}, 'parent_ids': ['8de8c306-23df-4653-ae5d-378a3564b494', 'e38d1fe9-0851-4e58-84f3-741b1ee0671c', '0e2f81bb-4883-4bf9-aa90-d46